# Tarea 3: Piping y Bombas
### Pablo Correa e Ian Gross

## Librerías

In [5]:
import CoolProp.CoolProp as cp
import fluids as fld
import numpy as np
import scipy.constants as cte
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize

from fluids.units import *

In [6]:
g = cte.g*u.m/u.s**2
Patm = (1*u.atm).to(u.Pa)

# Pregunta 1
Considere el sistema de almacenamiento de energía visto en el laboratorio.

A partir de los datos de operación del archivo anexo, verifique si la pérdida de carga observada se puede representar por la ecuación de Ergun.

¿Cuánto es el error con respecto a la correlación? (3pts), ¿Cuanto afecta la temperatura y la variabilidad de las propiedades termofísicas del aire? (3pts).

Asuma que el sistema de almacenamiento es un intercambiador de calor de flujo paralelo, donde el solido es uno de los medio de transferencia de calor. ¿Cuanto sería la efectividad del sistema (intercambiador de calor)? (3pts)

In [7]:
headers = ["Scan Sweep Time (s)", "Temp. Entrada (°C)", "Vel. Aire (m/s)", "Temp. Aire (°C)","Presión Dif. (mbar)", "Temp. Salida (°C)"]
data = pd.read_excel("Ensayo Piloto Lleno T-220°C.xlsx", skiprows=16, usecols="A,C:G", names=headers)

P_0 = 101325
T_0 = 273.15

P = pd.DataFrame(data['Presión Dif. (mbar)'] * 100, columns=["Presión Dif. (mbar)"])
P = P.rename(columns={"Presión Dif. (mbar)": "Presión Dif. (Pa)"})

rho = pd.DataFrame([cp.PropsSI('D', 'T', float(data['Temp. Aire (°C)'].iloc[i]) + T_0,
                               'P', float(P["Presión Dif. (Pa)"].iloc[i]) + P_0, 'Air')
                               for i in range(data.shape[0])], columns=["Rho (kg/m3)"])
mu = pd.DataFrame([cp.PropsSI('V', 'T', float(data['Temp. Aire (°C)'].iloc[i]) + T_0,
                               'D', float(rho['Rho (kg/m3)'].iloc[i]), 'Air') 
                               for i in range(data.shape[0])], columns=["Mu (Pa*s)"])

data = data.join([P, rho, mu]).drop(columns=["Presión Dif. (mbar)"])
data

,Scan Sweep Time (s),Temp. Entrada (°C),Vel. Aire (m/s),Temp. Aire (°C),Temp. Salida (°C),Presión Dif. (Pa),Rho (kg/m3),Mu (Pa*s)
0,2024-09-25 09:49:13.522,14.079405,6.472987,13.120901,17.538146,99.246850,1.234817,0.000018
1,2024-09-25 09:49:23.515,14.078369,6.656212,13.121559,17.564700,116.513805,1.235024,0.000018
2,2024-09-25 09:49:33.515,14.084538,6.437116,13.121187,17.573132,121.386540,1.235085,0.000018
3,2024-09-25 09:49:43.515,14.115000,6.546101,13.121459,17.570347,105.121852,1.234886,0.000018
4,2024-09-25 09:49:53.515,14.115085,6.437106,13.121394,17.589255,113.813643,1.234992,0.000018
...,...,...,...,...,...,...,...,...
2193,2024-09-25 15:54:43.516,30.401201,6.624077,18.110073,19.313422,-99.329177,1.211225,0.000018
2194,2024-09-25 15:54:53.516,30.379024,6.654350,17.866438,19.329506,-81.157420,1.212460,0.000018
2195,2024-09-25 15:55:03.516,30.373109,6.654364,18.111046,19.376573,-89.140490,1.211343,0.000018
2196,2024-09-25 15:55:13.516,30.396446,6.623994,18.111007,19.408551,-102.191115,1.211187,0.000018


Se sabe que la ecuación de Ergun es la siguiente:

$\frac{\Delta P}{L} = \frac{150 \mu (1-\epsilon)^2}{\phi^2 d^2_p \epsilon^3} v + \frac{1.75 \rho (1-\epsilon)}{\phi d_p \epsilon^3} v^2$

Despejando la presión diferencial se obtiene que:

$\Delta P = 150 \mu v \frac{L(1-\epsilon)^2}{\phi^2 d_p^2 \epsilon^3} + 1.75 \rho v^2 \frac{L(1-\epsilon)}{\phi d_p \epsilon^3}$

Luego, se optimiza para estimar los datos del lecho tal que minimice el error.

In [8]:
def ergun_loss(params, data):
    phi, dp, epsilon = params
    velocidad = np.array(data["Vel. Aire (m/s)"])
    delta_P_observada = np.array(data["Presión Dif. (Pa)"])
    rho = np.array(data["Rho (kg/m3)"])
    mu = np.array(data["Mu (Pa*s)"])
    L = 1.5
    
    # Pérdida de presión teórica según Ergun
    term_viscoso = (150 * mu * (1 - epsilon)**2) / (phi**2 * dp**2 * epsilon**3)
    term_inercial = (1.75 * rho * (1 - epsilon)) / (phi * dp * epsilon**3)
    delta_P_calculada = L * (term_viscoso * velocidad + term_inercial * velocidad**2)
    
    # Error entre valores calculados y observados
    error = np.sum((delta_P_calculada - delta_P_observada)**2)
    return error

params_iniciales = [0.8, 0.05, 0.4]
bounds = [(0.1, 1.0),   # phi
          (0.001, 0.1), # dp
          (0.5, 0.9)]  # epsilon
resultado = minimize(ergun_loss, params_iniciales, args=data, bounds=bounds)
print(resultado)


  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 102388733.19860311
        x: [ 1.000e+00  1.000e-01  9.000e-01]
      nit: 1
      jac: [-1.571e+08 -1.571e+09 -2.095e+09]
     nfev: 8
     njev: 2
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>


Teniendo los datos estimados se compara con la presión experimental el cálculo de la ecuación de Ergun

In [9]:
phi, dp, epsilon = resultado["x"]
L = 1.5
    
# Pérdida de presión teórica según Ergun
term_viscoso = (150 * data["Mu (Pa*s)"] * (1 - epsilon)**2) / (phi**2 * dp**2 * epsilon**3)
term_inercial = (1.75 * data["Rho (kg/m3)"] * (1 - epsilon)) / (phi * dp * epsilon**3)
delta_P_calculada = L * (term_viscoso * data["Vel. Aire (m/s)"] + term_inercial * data["Vel. Aire (m/s)"]**2)
# delta_P_calculada = pd.DataFrame(delta_P_calculada).rename(["Presión Teórica (Pa)"])
data = data.join(pd.DataFrame(delta_P_calculada, columns=["Presión Teórica (Pa)"]))
data = data.join(pd.DataFrame(abs(data["Presión Dif. (Pa)"]-data["Presión Teórica (Pa)"])/data["Presión Dif. (Pa)"], columns=["Error Relativo"]))
data

,Scan Sweep Time (s),Temp. Entrada (°C),Vel. Aire (m/s),Temp. Aire (°C),Temp. Salida (°C),Presión Dif. (Pa),Rho (kg/m3),Mu (Pa*s),Presión Teórica (Pa),Error Relativo
0,2024-09-25 09:49:13.522,14.079405,6.472987,13.120901,17.538146,99.246850,1.234817,0.000018,186.336114,0.877502
1,2024-09-25 09:49:23.515,14.078369,6.656212,13.121559,17.564700,116.513805,1.235024,0.000018,197.066389,0.691357
2,2024-09-25 09:49:33.515,14.084538,6.437116,13.121187,17.573132,121.386540,1.235085,0.000018,184.316885,0.518429
3,2024-09-25 09:49:43.515,14.115000,6.546101,13.121459,17.570347,105.121852,1.234886,0.000018,190.579564,0.812940
4,2024-09-25 09:49:53.515,14.115085,6.437106,13.121394,17.589255,113.813643,1.234992,0.000018,184.302411,0.619335
...,...,...,...,...,...,...,...,...,...,...
2193,2024-09-25 15:54:43.516,30.401201,6.624077,18.110073,19.313422,-99.329177,1.211225,0.000018,191.408603,-2.927013
2194,2024-09-25 15:54:53.516,30.379024,6.654350,17.866438,19.329506,-81.157420,1.212460,0.000018,193.358759,-3.382515
2195,2024-09-25 15:55:03.516,30.373109,6.654364,18.111046,19.376573,-89.140490,1.211343,0.000018,193.181540,-3.167158
2196,2024-09-25 15:55:13.516,30.396446,6.623994,18.111007,19.408551,-102.191115,1.211187,0.000018,191.397772,-2.872939


In [10]:
np.sum(data["Error Relativo"])/data.shape[0]

np.float64(-0.8925577726707616)

Se nota que existe una muy gran diferencia entre los valores teóricos y los experimentales, por lo que la ecaución no es una buena aproximación.

In [11]:
data.corr()["Presión Dif. (Pa)"]

Scan Sweep Time (s)    -0.822679
Temp. Entrada (°C)      0.665450
Vel. Aire (m/s)         0.820089
Temp. Aire (°C)         0.963905
Temp. Salida (°C)       0.736789
Presión Dif. (Pa)       1.000000
Rho (kg/m3)            -0.963999
Mu (Pa*s)               0.964242
Presión Teórica (Pa)    0.077703
Error Relativo          0.951328
Name: Presión Dif. (Pa), dtype: float64

Observando las correlaciones, se nota que el factor que tiene la mayor incidencia en el resultado en la presión es la temperatura del aire, y las propiedades asociadas (densidad y viscosidad).

El factor menos incidente es la temperatura de entrada.

# Pregunta 2
Liquid  carbon  dioxide  enters  the  tubes  of  a  shell  and  tube  heat  exchanger  at  a  temperature of 0°C and a flow rate of 30 kg/s. The carbon dioxide is heated with a flow of city water that enters the shell side of the heat exchanger at a temperature of 40°C and a flow rate of 31 kg/s. The heat exchanger is a single shell, two-tube pass configuration with a 25in shell. The tubes are ¾in 10BWG laid out on a 1in triangular pitch. The heat exchanger is 2m long and contains 2 baffles per meter of length.
* Asumiendo presión atmosférica en la entrada
* Asumiendo que las paredes del intercambiador son de hierro forjado. [1]
* Asumiendo que el cambio de $\mu$ es despreciable entre la coraza y la pared del tubo

In [12]:
# Temperaturas
T_CO2 = (0*u.degC).to(u.K)
T_CityWater = (40*u.degC).to(u.K)

# Masas
dm_CO2 = 30*u.kg/u.s
dm_CityWater = 31*u.kg/u.s

# Diametros
d_shell = (25*u.inch).to(u.m)
do_tube = (0.75*u.inch).to(u.m)
t_tube = t_from_gauge(10, schedule='BWG')
di_tube = do_tube - t_tube

# Longitudes
l = 2*u.m
Pt = (1*u.inch).to(u.m)
B = 0.5*u.m

# Número de tubos
N_T = 452*u.dimensionless

# Número de pasos
N_P = 2*u.dimensionless

# Número de bafles
N_B = 4*u.dimensionless

# Fouling Factor
R_CO2 = 0.000176*u.m**2*u.K/u.W
R_CityWater = 0.000352*u.m**2*u.K/u.W

In [13]:
# Diámetro hidáulico dado en clases
d_shell_eq = 2*np.sqrt(3)*Pt**2/(np.pi*do_tube)-do_tube
print(f"d_shell_eq = {d_shell_eq:.4f}")

# Área efectiva de transferencia de calor
A_tube_eff = N_T/N_P*(np.pi*di_tube**2/4)
print(f"A_tube_eff = {A_tube_eff:.4f}")

# Área transversal de un tubo
A_tube_cross = np.pi * di_tube**2/4*N_T

# Area efectiva de transferencia de calor
A_shell_eff = (d_shell*(Pt-do_tube)*B/Pt).to(u.m**2)
print(f"A_shell_eff = {A_shell_eff:.4f}")



d_shell_eq = 0.0183 meter
A_tube_eff = 0.0435 meter ** 2
A_shell_eff = 0.0794 meter ** 2


In [14]:
# Propiedades de los fluidos
rho_CO2 = cp.PropsSI('D', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.kg/u.m**3
mu_CO2 = cp.PropsSI('V', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.Pa*u.s
k_CO2 = cp.PropsSI('L', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.W/u.m/u.K
cp_CO2 = cp.PropsSI('Cpmass', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.J/u.kg/u.K


rho_CityWater = cp.PropsSI('D', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.kg/u.m**3
mu_CityWater = cp.PropsSI('V', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.Pa*u.s
k_CityWater = cp.PropsSI('L', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.W/u.m/u.K
cp_CityWater = cp.PropsSI('Cpmass', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.J/u.kg/u.K

# Cp
C_CO2 = dm_CO2*cp_CO2
C_CityWater = dm_CityWater*cp_CityWater
Cmin = min(C_CO2, C_CityWater)
Cmax = max(C_CO2, C_CityWater)
Cr = Cmin/Cmax
print(f"Cr = {Cr:.2f}")

Cr = 0.59 dimensionless


### Calculando h_interno

In [15]:
# Condiciones para Nusselt
V_CO2 = dm_CO2/(rho_CO2*A_tube_eff)

Re_CO2 = fld.core.Reynolds(V_CO2, di_tube, rho_CO2, mu_CO2).to(u.dimensionless)
print(f"Re_CO2 = {Re_CO2:.2f}")
Pr_CO2 = fld.core.Prandtl(Cp=cp_CO2, mu=mu_CO2, k=k_CO2).to(u.dimensionless)
print(f"Pr_CO2 = {Pr_CO2:.2f}")

print(f"L/D = {l/di_tube:.2f}")

Re_CO2 = 107590.86 dimensionless
Pr_CO2 = 2.34 dimensionless
L/D = 127.83 dimensionless


Para calcular el número de Nusselt dado que el flujo es turbulento y se está calentando:

In [16]:
Nu_CO2 = 0.023*Re_CO2**0.8*Pr_CO2**0.4
print(f"Nu_CO2 = {Nu_CO2:.2f}")

h_tube = (k_CO2*Nu_CO2/t_tube).to(u.W/u.m**2/u.K)
print(f"h_tube = {h_tube:.2f}\n")


Nu_CO2 = 342.55 dimensionless
h_tube = 10983.97 watt / kelvin / meter ** 2



### Calculando R de conducción

In [17]:
k = 45*u.W/u.K/u.m
Rw = np.log(do_tube/di_tube)/(2*np.pi*k*l)/N_T
print(f"Rw = {Rw:.5}\n")

Rw = 7.7016e-07 kelvin / watt



### Calculando h_externo

In [18]:
V_shell = dm_CityWater/(rho_CityWater*A_shell_eff)
print(f"Vs = {V_shell:.3f}")

Re_CityWater = fld.core.Reynolds(V_shell, d_shell_eq, rho_CityWater, mu_CityWater).to(u.dimensionless)
print(f"Re_CityWater = {Re_CityWater:.2f}")

Pr_CityWater = fld.core.Prandtl(Cp=cp_CityWater, mu=mu_CityWater, k=k_CityWater).to(u.dimensionless)
print(f"Pr_CityWater = {Pr_CityWater:.2f}")

Vs = 0.394 meter / second
Re_CityWater = 10945.57 dimensionless
Pr_CityWater = 4.34 dimensionless


Para calcular el número de Nusselt dado que el flujo es turbulento y se está enfriando:

In [19]:
Nu_CityWater = 0.36*Re_CityWater**0.55*Pr_CityWater**0.33
print(f"Nu_CityWater = {Nu_CityWater:.2f}")

h_shell = (k_CityWater*Nu_CityWater/d_shell_eq).to(u.W/u.m**2/u.K)
print(f"h_shell = {h_shell:.2f}\n")

Nu_CityWater = 97.34 dimensionless
h_shell = 3344.09 watt / kelvin / meter ** 2



### Cálculo de  y NTU

In [20]:
UA = ((1/(h_tube*A_tube_eff) + Rw + 1/(h_shell*A_shell_eff))**-1).to(u.W/u.K)
print(f"UA = {UA:.2f}")
NTU = (UA/Cmin).to(u.dimensionless)
print(f"NTU = {NTU}")


UA = 170.55 watt / kelvin
NTU = 0.002236153515557553 dimensionless


### 2.a

In [21]:
# Iterando
T_CO2_out = (13.607*u.degC).to(u.K)
T_CityWater_out = (31.99*u.degC).to(u.K)
print(f"T_CO2_out = {T_CO2_out.to(u.degC):.2f}")
print(f"T_CityWater_out = {T_CityWater_out.to(u.degC):.2f}\n")

Pot_CO2 = (C_CO2*(T_CO2_out-T_CO2)).to(u.W)
Pot_CityWater = (C_CityWater*(T_CityWater_out-T_CityWater)).to(u.W)
print(f"Pot_CO2 = {Pot_CO2}")
print(f"Pot_CityWater = {Pot_CityWater}")
print(f"DPot = {Pot_CO2+Pot_CityWater}\n")

R = C_CO2/C_CityWater
P = (T_CO2_out-T_CO2)/(T_CityWater-T_CO2)
F = (np.sqrt(R**2+1)*np.log((1-P)/(1-P*R)))/((R-1)*np.log((2-P*(R+1-np.sqrt(R**2+1)))/(2-P*(R+1+np.sqrt(R**2+1))))).to(u.dimensionless)
print(f"F = {F:.2f}\n")

Q = UA*F*(T_CityWater-T_CityWater_out + T_CO2_out-T_CO2)/np.log((T_CO2_out-T_CO2)/(T_CityWater-T_CityWater_out))
Qmax = (Cmin*(T_CityWater-T_CO2)).to(u.W)
print(f"Q = {Q:.2f}")
print(f"Qmax = {Qmax:.2f}\n")
print(f"Q/Qmax = {(Q/Qmax*100).magnitude}")

e = (2*(1+Cr+np.sqrt(1+Cr**2)*(1+np.exp(-NTU*np.sqrt(1+Cr**2)))/(1-np.exp(-NTU*np.sqrt(1+Cr**2))))**-1*100).magnitude
print(f"Efficiency = {e}")

T_CO2_out = 13.61 degree_Celsius
T_CityWater_out = 31.99 degree_Celsius

Pot_CO2 = 1037796.4631310743 watt
Pot_CityWater = -1037790.4884946353 watt
DPot = 5.974636439001188 watt

F = 0.98 dimensionless

Q = 6805.41 watt
Qmax = 3050772.29 watt

Q/Qmax = 0.22307155741650378
Efficiency = 0.22321873267121367


Se obtienen unas temperaturas de salida de T_CO2_out = 13.31°C y T_CityWater_out = 31.99°C

Un error en diferencia de potencia de 5.97W

Y un error de eficiencia de 0.0002

In [22]:
# Pérdida de carga en los tubos
e = 0.045*u.mm
f_CO2 = fld.friction.friction_factor(Re_CO2, eD=e/di_tube).to(u.dimensionless)
DP_CO2 = N_P*(f_CO2*l*V_CO2**2*rho_CO2/(2*di_tube)).to(u.kPa)
DPm_CO2 = (4*N_P*V_CO2**2*rho_CO2/2).to(u.kPa)
print(f"f_CO2 = {f_CO2}")
print(f"DP_CO2 = {DP_CO2}")
print(f"DPm_CO2 = {DPm_CO2}")
print(f"DPt_CO2 = {DP_CO2 + DPm_CO2}\n")

f_CityWater = np.exp(0.576-0.19*np.log(Re_CityWater))
DP_CityWater = (f_CityWater*(N_B+1)*d_shell/d_shell_eq*V_shell**2*rho_CityWater/2).to(u.kPa)
print(f"f_CityWater = {f_CityWater}")
print(f"DP_CityWater = {DP_CityWater}\n")

print(f"DPt = {DP_CO2 + DPm_CO2 + DP_CityWater}")

f_CO2 = 0.027110861016610137 dimensionless
DP_CO2 = 1.7812320095329095 kilopascal
DPm_CO2 = 2.0559403114550414 kilopascal
DPt_CO2 = 3.8371723209879507 kilopascal

f_CityWater = 0.3038773563385135 dimensionless
DP_CityWater = 4.053857761401604 kilopascal

DPt = 7.8910300823895545 kilopascal


Se obtiene una pérdida de carga total de 7.89 kilo pascales.

### b. The outlet temperatures of each fluid after the heat exchanger has been in continuous service for one year without maintenancece.

Se considera el factor de ensuciamiento de la coraza y de los tubos como sigue:
$$R^"_{f,c} = \frac{1}{U_d} - \frac{1}{U}$$
$$U_d = \bigg(\frac{1}{U} +R^"_{f,c}\bigg)^{-1}$$

In [23]:
UdA = (UA**-1+R_CO2/A_shell_eff+R_CityWater/A_shell_eff)**-1
print(f"UdA = {UdA:.2f}")
NTUd = (UdA/Cmin).to(u.dimensionless)
print(f"NTUd = {NTUd}")

UdA = 79.90 watt / kelvin
NTUd = 0.0010476277582224453 dimensionless


In [34]:
# Iterando
T_CO2_out_d = (6.2685*u.degC).to(u.K)
T_CityWater_out_d = (36.31*u.degC).to(u.K)
print(f"T_CO2_out = {T_CO2_out_d.to(u.degC)}")
print(f"T_CityWater_out = {T_CityWater_out_d.to(u.degC)}\n")

Pot_CO2 = (C_CO2*(T_CO2_out_d-T_CO2)).to(u.W)
Pot_CityWater = (C_CityWater*(T_CityWater_out_d-T_CityWater)).to(u.W)
print(f"Pot_CO2 = {Pot_CO2}")
print(f"Pot_CityWater = {Pot_CityWater}")
print(f"DPot = {Pot_CO2+Pot_CityWater}\n")

R = C_CO2/C_CityWater
P = (T_CO2_out_d-T_CO2)/(T_CityWater-T_CO2)
F = (np.sqrt(R**2+1)*np.log((1-P)/(1-P*R)))/((R-1)*np.log((2-P*(R+1-np.sqrt(R**2+1)))/(2-P*(R+1+np.sqrt(R**2+1))))).to(u.dimensionless)
print(f"F = {F}\n")

Q = UA*F*(T_CityWater-T_CityWater_out_d + T_CO2_out_d-T_CO2)/np.log((T_CO2_out_d-T_CO2)/(T_CityWater-T_CityWater_out_d))
Qmax = (Cmin*(T_CityWater-T_CO2)).to(u.W)
print(f"Q = {Q:.2f}")
print(f"Qmax = {Qmax:.2f}\n")
print(f"Q/Qmax = {(Q/Qmax*100).magnitude}")

e = (2*(1+Cr+np.sqrt(1+Cr**2)*(1+np.exp(-NTUd*np.sqrt(1+Cr**2)))/(1-np.exp(-NTUd*np.sqrt(1+Cr**2))))**-1*100).magnitude
print(f"Efficiency = {e}")

T_CO2_out = 6.268500000000017 degree_Celsius
T_CityWater_out = 36.31 degree_Celsius

Pot_CO2 = 478094.1522111539 watt
Pot_CityWater = -478083.2587447199 watt
DPot = 10.893466433975846 watt

F = 0.9968449450424183 dimensionless

Q = 3195.00 watt
Qmax = 3050772.29 watt

Q/Qmax = 0.10472744977336525
Efficiency = 0.10467565520953458


Se obtienen unas temperaturas de salida de T_CO2_out = 6.27°C y T_CityWater_out = 36.31°C

Un error en diferencia de potencia de 10.89W

Y un error de eficiencia de 0.0001

El valor obtenido es considerablemente peor que el de un sistema nuevo, lo cual tiene sentido físico.

c. If the outlet temperature of the carbon dioxide must be 20°C or higher, is this heat exchanger sufficient for the required duty?

El dióxido de carbono no llega a los 20°C incluso cuando el sistema está nuevo, por lo tanto, el intercambiador de calor no es suficiente para realizar el trabajo requerido.

# Pregunta 3
Considere el plano hidráulico de la localidad de Hualañé y la distribución de diámetros determinada en la Tarea 2. Seleccione una bomba de uno de los dos catalogos disponibles, que permita entregar el caudal y nivel de presión requerido (6pts). Identifique el punto de operación de la bomba (6pts)

# Bibliografía

[1] https://www.stainless-structurals.com/blog/comparing-the-thermal-conductivity-of-stainless-steel-to-other-metals/